### This notebook creates a trigram model for the generation of new names.  
We do this as follows:
1. Read in the names from a list of names.
2. Generate a dictionary to get a mapping from chars to indexes and vice versa.
3. Create the list of all trigrams of words in the name list.
4. Create a tensor N to count the occurences of each trigram
5. Turn this tensor into a tensor P, which counts the relative frequencies of each third char, given two previous ones.
6. Use this tensor in order to sample new chars and generate new names.
7. Calculate the average negative likelihood of each word in the test set.

Since we used the relative frequencies of the trigrams in the training set as our probabilities to sample new chars, our model is optimal wrt. to minimizing the expected NLL among all models, which only consider trigrams.

In [175]:
#read in the names from the files
f = open("names.txt", "r")
words = f.read().splitlines()

#get chars and a dictionary converting chars to numbers and vice versa
chars = sorted(list(set("".join(words) )))
stoi = {s : i+1 for i,s in enumerate(chars)}
stoi["."] = 0
itos = {i: s for s,i in stoi.items()}

In [333]:
#build up the dataset of all trigrams
x = []
x_test = []
split = int(len(words)*0.8)

for w in words[:split]:
    context = [0]*3
    
    for char in w + ".":
        ix = stoi[char]
        context = context[1:] + [ix]
        x.append(context)

for w in words[split+1:]:
    context = [0]*3
    
    for char in w + ".":
        ix = stoi[char]
        context = context[1:] + [ix]
        x_test.append(context)


In [335]:
x[:20]

[[0, 0, 5],
 [0, 5, 13],
 [5, 13, 13],
 [13, 13, 1],
 [13, 1, 0],
 [0, 0, 15],
 [0, 15, 12],
 [15, 12, 9],
 [12, 9, 22],
 [9, 22, 9],
 [22, 9, 1],
 [9, 1, 0],
 [0, 0, 1],
 [0, 1, 22],
 [1, 22, 1],
 [22, 1, 0],
 [0, 0, 9],
 [0, 9, 19],
 [9, 19, 1],
 [19, 1, 2]]

In [390]:
import torch
#get a tensor which saves the number of occurences of each 3-gram in the dataset
#smooth the tensor out by adding 1 to each 3-gram count

N = torch.ones(27,27,27)

for count in x:
    a,b,c = count
    N[a,b,c] += 1

In [391]:
#we set all counts of three consecutive dots to zero.
N[0,0,0]=0

#in order to samle from N, we convert it into relative frequencies, by broadcasting
P = N / N.sum(dim = 2, keepdim = True)

#get rid of nan in the first row, created by division by 0
P[0,0,0]=0

In [392]:
#check that we calculated relative frequencies
#all entries are 1, except for the ones we manually set to zero
P.sum(dim = 2)

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000,

In [396]:
#set seed and test sampling from the trigram frequencies.
g = torch.Generator().manual_seed(1)

ix = torch.multinomial(P[1,14,:], num_samples = 100, replacement = True , generator = g)

bi = itos[1] + itos[14]
trigrams = []
for i in ix:
    trigrams.append(bi + itos[i.item()])
print(trigrams)    

['an.', 'ana', 'ana', 'an.', 'ani', 'an.', 'and', 'ann', 'ani', 'ane', 'ane', 'ana', 'ann', 'ana', 'an.', 'an.', 'ani', 'an.', 'ane', 'ann', 'ant', 'anz', 'ano', 'ann', 'ann', 'an.', 'an.', 'any', 'ann', 'and', 'and', 'and', 'ann', 'ant', 'any', 'an.', 'anv', 'ana', 'and', 'ana', 'an.', 'ana', 'ana', 'and', 'anc', 'ana', 'ann', 'ana', 'ana', 'ana', 'ani', 'ana', 'and', 'ann', 'ann', 'ana', 'ann', 'ane', 'anq', 'an.', 'ani', 'any', 'an.', 'ana', 'ana', 'ane', 'and', 'ane', 'ang', 'ana', 'an.', 'ani', 'ana', 'ang', 'and', 'ani', 'ang', 'ana', 'ani', 'ann', 'an.', 'ana', 'ant', 'anh', 'ann', 'ann', 'ann', 'any', 'ann', 'ana', 'anz', 'ani', 'and', 'ang', 'ann', 'and', 'an.', 'ano', 'ann', 'ans']


In [398]:
from collections import Counter

Counter(trigrams)

Counter({'ana': 21,
         'ann': 18,
         'an.': 15,
         'and': 11,
         'ani': 9,
         'ane': 6,
         'any': 4,
         'ang': 4,
         'ant': 3,
         'anz': 2,
         'ano': 2,
         'anv': 1,
         'anc': 1,
         'anq': 1,
         'anh': 1,
         'ans': 1})

In [400]:
#generate a few names

for i in range(20):
    out = ""
    a, b = 0, 0
    
    while True:
        #sample next char
        sample = torch.multinomial(P[a,b,:], num_samples = 1, generator = g).item()
        out += itos[sample]
        #update previous trigrams
        a = b
        b = sample
        #stop when a dot is sampled
        if sample == 0: 
            break
    print(out)

anne.
de.
jew.
prnbff.
guah.
da.
arihyaa.
alexya.
jedyla.
kimitiandrissjkxpgiustoni.
aanny.
ne.
agana.
asmeyviahmiva.
leanier.
tapb.
rulei.
on.
rosellyn.
emmaxib.


To compute a loss for our model, we consider the likelihood of each word of the test set.
This is the product of the probabilities of each trigram.

For example, given the name "Anna", 
$$ \text{Lik}(\text{"Anna"}) = p(..a)\cdot p(.an)\cdot p(ann)\cdot p(nna) \cdot p(na.) $$
Maximizing this likelihood is equivalent to minimizing the log likelihood, which is again equivalent to maximizing the negative log likelihood.
We compute the NLL for the entire test dataset and divide it by the number of words in the dataset to get the average NLL per word.
The average NLL can also serve as a loss function for our model.

Because our trigram model assigns to each trigram its relative frequency among the training set, it is the maximum likelihood estimator for all models considering only trigrams.
That is to say, that among all models, which only consider trigrams, our trigram model (almost) minimizes the expected NLL of the words in the training set.  
The model only almost minimizes the expected NLL, because we smoothed the occurence matrix N, by adding 1 to each trigram occurence.

In [404]:
#next we compute the loss of the trigram model.
#to do so, we calculate the likelihood of each word occuring, based on the trigram frequencies.
#as a loss function we take the NLL of each word of the test set.
import numpy as np

nll = 0
for tri in x_test:
    a,b,c = tri
    nll -= np.log(P[a,b,c])
    
nll /= len(words)-split
nll.item()



17.18137550354004